<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [11]:
import chess.pgn
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.animation as animation
import time
import colorcet as cc

In [30]:
pgn.close()

In [27]:
def animateGame(game):
    fig = plt.gcf()
    ax = plt.gca()
    colormap = plt.cm.get_cmap("Greys_r")
    board = game.board()
    
    data = getPresenceMatrix(board)
    heatmap = ax.pcolor(data, cmap=colormap)
    fig.canvas.draw()
    fig.show()
    
    for move in game.mainline_moves():
        time.sleep(0.3)
        board.push(move)
        data = getPresenceMatrix(board)
        heatmap = ax.pcolor(data, cmap=colormap)
        ax.draw_artist(ax.patch)
        ax.draw_artist(heatmap)
        fig.canvas.blit(ax.bbox)
        fig.canvas.flush_events()

pgn = open("../Datafiles/PGN/byrne_fischer_1956.pgn")
game = chess.pgn.read_game(pgn)   
animateGame(game)

In [3]:
blackPieces = ['p','r','b','n','k','q']
whitePieces = ['P','R','B','N','K','Q']


def getPresenceMatrix(board):
    matrix = np.zeros((8,8), dtype=np.int)
    for file in range (0,8):
        for rank in range (0,8):
            square = (file)*8 + (rank)
            #print(board.piece_at(square), square)
            if(board.piece_at(square) != None):
                if(board.piece_at(square).symbol() in whitePieces):
                    matrix[file,rank] = matrix[file, rank] + 1
                    #print("+1")
                elif(board.piece_at(square).symbol() in blackPieces):
                    matrix[file,rank] = matrix[file, rank] - 1
                    #print("-1")
    return matrix
    

def getAttackingSquaresPawn(square, white):
    rank = chess.square_rank(square)
    file = chess.square_file(square)
    if(white):
        if(file == 0):
            return [chess.square(1, rank+1)]
        elif(file == 7):
            return [chess.square(6, rank+1)]
        else:
            return [chess.square(file-1, rank+1), chess.square(file+1, rank+1)]
    else:
        if(file == 0):
            return [chess.square(1, rank-1)]
        elif(file == 7):
            return [chess.square(6, rank-1)]
        else:
            return [chess.square(file-1, rank-1), chess.square(file+1, rank-1)]
            
            
     
    
def getAttackingMatrix(board):
    matrix = np.zeros((8,8), dtype=np.int)
    legalMoves = board.pseudo_legal_moves
    attackingSquaresWhite = []
    attackingSquaresBlack = []

    #get attacking squares for player to move
    if(board.turn):
        for move in legalMoves:
            toRank = chess.square_rank(move.to_square)
            fromRank = chess.square_rank(move.from_square)
            if(board.piece_at(move.from_square).piece_type == chess.PAWN and abs(toRank - fromRank) == 1):
                attackingSquaresWhite = attackingSquaresWhite + getAttackingSquaresPawn(move.from_square, True)
            elif(board.piece_at(move.from_square).piece_type != chess.PAWN):
                attackingSquaresWhite.append(move.to_square)
    else:
        for move in legalMoves:
            toRank = chess.square_rank(move.to_square)
            fromRank = chess.square_rank(move.from_square)
            if(board.piece_at(move.from_square).piece_type == chess.PAWN and abs(toRank - fromRank) == 1):
                attackingSquaresBlack = attackingSquaresBlack + getAttackingSquaresPawn(move.from_square, False)
            elif(board.piece_at(move.from_square).piece_type != chess.PAWN):
                attackingSquaresBlack.append(move.to_square)
                
    #get attacking squares for other player
    board.push(chess.Move.from_uci("0000"))
    legalMoves = board.pseudo_legal_moves
    if(board.turn):
        for move in legalMoves:
            toRank = chess.square_rank(move.to_square)
            fromRank = chess.square_rank(move.from_square)
            if(board.piece_at(move.from_square).piece_type == chess.PAWN and abs(toRank - fromRank) == 1):
                attackingSquaresWhite = attackingSquaresWhite + getAttackingSquaresPawn(move.from_square, True)
            elif(board.piece_at(move.from_square).piece_type != chess.PAWN):
                attackingSquaresWhite.append(move.to_square)
    else:
        for move in legalMoves:
            toRank = chess.square_rank(move.to_square)
            fromRank = chess.square_rank(move.from_square)
            if(board.piece_at(move.from_square).piece_type == chess.PAWN and abs(toRank - fromRank) == 1):
                attackingSquaresBlack = attackingSquaresBlack + getAttackingSquaresPawn(move.from_square, False)
            elif(board.piece_at(move.from_square).piece_type != chess.PAWN):
                attackingSquaresBlack.append(move.to_square)
                
    
    #Fill in matrix
    for square in attackingSquaresWhite:
        rank = chess.square_rank(square)
        file = chess.square_file(square)
        matrix[rank,file] = matrix[rank,file] + 1
    for square in attackingSquaresBlack:
        rank = chess.square_rank(square)
        file = chess.square_file(square)
        matrix[rank,file] = matrix[rank,file] - 1
        
    return matrix
        
    
print(getAttackingMatrix(chess.Board()))


[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 2  2  3  2  2  3  2  2]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [-2 -2 -3 -2 -2 -3 -2 -2]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
